In [ ]:
# clone codes
import os
os.chdir('/content/')
!git clone https://github.com/neuphonic/neutts-air



In [ ]:
# prepare benchmark
!git clone https://github.com/skiloop/tts-benchmarks
os.chdir('/content/tts-benchmarks/')
!DST_PATH=~/.local/bin/ sh install.sh


In [ ]:
# test and prepare models
import os
os.chdir('/content/neutts-air/')

!sh /content/tts-benchmarks/prepare-models.sh

In [ ]:
# config performance test
import json
import os

# Define test parameters
TEST_TEXT = "My name is Dave, and um, I'm from London"
REF_AUDIO = "./samples/dave.wav"
REF_TEXT = "./samples/dave.txt"
REF_CODES = "samples/dave.pt"

# Build configuration using variables
config = {
  "description": "TTS性能测试",
  "iterations": 5,
  "tests": [
    {
      "name": "neutts",
      "command": f"conda activate neutts-air && python -m examples.basic_example --input_text \"{TEST_TEXT}\" --ref_audio {REF_AUDIO} --ref_text {REF_TEXT}"
    },
    {
      "name": "neutts-gguf",
      "command": f"conda activate neutts-air && python -m examples.basic_example --input_text \"{TEST_TEXT}\" --ref_audio {REF_AUDIO} --ref_text {REF_TEXT} --backbone neuphonic/neutts-air-q4-gguf"
    },
    {
      "name": "neutts-onnx-gguf",
      "command": f"conda activate neutts-air && python -m examples.onnx_example --input_text \"{TEST_TEXT}\" --ref_text {REF_TEXT} --ref_codes {REF_CODES} --backbone neuphonic/neutts-air-q4-gguf"
    },
    {
      "name": "neutts-onnx",
      "command": f"conda activate neutts-air && python -m examples.onnx_example --input_text \"{TEST_TEXT}\" --ref_text {REF_TEXT} --ref_codes {REF_CODES} --backbone neuphonic/neutts-air-q4-gguf"
    },
    {
      "name": "edge-tts",
      "command": f"conda activate neutts-air && edge-tts --text \"{TEST_TEXT}\" --write-media ./outputs/dave-edge-tts.wav"
    },
    {
      "name": "chartts",
      "command": f"conda activate chartts && chartts -t \"{TEST_TEXT}\" -o ./outputs/dave-chartts.wav"
    },
    {
      "name": "coqui-tts",
      "command": f"conda activate coqui-tts && tts --text \"{TEST_TEXT}\" --out_path ./outputs/dave-coqui-tts.wav"
    }
  ]
}

with open('perftest.json', 'w') as f:
  json.dump(config, f, indent=2)


In [ ]:
# do performance test
!perftest -f perftest.json -o pefromance.json
